In [ ]:
# Imports

import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import csv
from random import randint

import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from skimage.transform import resize
from tensorflow import keras
from tensorflow.keras import Input, layers
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization, add
from keras.layers.core import Activation, Dropout, Flatten, Dense
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm
from PIL import Image

%matplotlib inline

In [ ]:
import pandas as pd
df1 = pd.read_csv('../input/old-gt-coords/old_left_eye.csv')
df2 = pd.read_csv('../input/old-gt-coords/old_right_eye.csv')

assert df1.shape == df2.shape
df1.shape

In [ ]:
import cv2
left = []
right = []
for index in range(170926):
    img_name = df1.iloc[index]['img_path'][7:-2]
    if(img_name != "p07-day17-152"):
        left_eye = cv2.imread('../input/segmented-mpiigaze/left/' + img_name + '-l.jpg')
        left_eye = cv2.cvtColor(left_eye, cv2.COLOR_BGR2RGB)
        right_eye = cv2.imread('../input/segmented-mpiigaze/right/' + img_name + '-r.jpg')
        right_eye = cv2.cvtColor(right_eye, cv2.COLOR_BGR2RGB)
        left.append(left_eye)
        right.append(right_eye)
left = np.array(left)
right = np.array(right)

test_left = []
test_right = []
for index in range(170926, 213657):
    img_name = df1.iloc[index]['img_path'][7:-2]
    if(img_name != "p07-day17-152"):
        left_eye = cv2.imread('../input/segmented-mpiigaze/left/' + img_name + '-l.jpg')
        left_eye = cv2.cvtColor(left_eye, cv2.COLOR_BGR2RGB)
        right_eye = cv2.imread('../input/segmented-mpiigaze/right/' + img_name + '-r.jpg')
        right_eye = cv2.cvtColor(right_eye, cv2.COLOR_BGR2RGB)
        test_left.append(left_eye)
        test_right.append(right_eye)
    
test_left = np.array(test_left)
test_right = np.array(test_right)

In [ ]:
from operator import itemgetter

labels = []
for i in range(170926):
    name = df1.iloc[i]['img_path'][7:-2]
    if(name != "p07-day17-152"):
        a = df1.iloc[i]
        b = df2.iloc[i]
        a_x = a['x_coord']
        a_y = a['y_coord']
        b_x = b['x_coord']
        b_y = b['y_coord']
        labels.append([(a_x+b_x)/2, (a_y+b_y)/2])
labels = np.array(labels)
labels.shape

test_labels = []

for i in range(170926, 213657):
    a = df1.iloc[i]
    b = df2.iloc[i]
    a_x = a['x_coord']
    a_y = a['y_coord']
    b_x = b['x_coord']
    b_y = b['y_coord']
    test_labels.append([(a_x+b_x)/2, (a_y+b_y)/2])

test_labels = np.array(test_labels)
test_labels.shape

print(len(labels))

In [ ]:
max_x_train = max(labels, key=itemgetter(0))[0]
max_x_test = max(test_labels, key=itemgetter(0))[0]
max_y_train = max(labels, key=itemgetter(1))[1]
max_y_test = max(test_labels, key=itemgetter(1))[1]
print(max_x_train)
print(max_x_test)
print(max_y_train)
print(max_y_test)

In [ ]:
for l in labels:
    l[0] = (l[0] / max_x_train) * 10
    l[1] = (l[1] / max_y_train) * 10

for l in test_labels:
    l[0] = (l[0] / max_x_test) * 10
    l[1] = (l[1] / max_y_test) * 10

In [ ]:
def create_model():
    IMAGE_SIZE = (36, 60)
    pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    x = tf.keras.layers.Input(shape=[*IMAGE_SIZE, 3]) # input is 192x192 pixels RGB
    pretrained_model.trainable = True

    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(5, activation='softmax', dtype=tf.float32) # the float32 is needed on softmax layer when using mixed precision
    ])

    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']
    )

    return x, model

x1, model1 = create_model()
x2, model2 = create_model()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import AvgPool2D, Concatenate, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D, ReLU

inp1, network1 = x1, model1
inp2, network2 = x2, model2
embedding = Concatenate()([network1.output,  network2.output])
d1 = Dense(15, activation='relu')(embedding)
drop = Dropout(.25)(d1)
output = Dense(2)(drop)

# model = Model(inputs=[inp1, inp2], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
history = model.fit(x = [left], y = labels, validation_data = ([test_left], test_labels), epochs=40, batch_size=32)

In [ ]:
model.save("./final_60_36.h5")